# Test för att göra zoombara bilder med event handling
Detta är ett test där vi undersöker hur man kan använda javascript-callbacks med bokeh. Detta programmet ska skriva ut positionen man klickar på i figuren till konsolen. Men det tenderar att krascha på grund av för hög upplösning. 

Importera paket och läs in data:

In [0]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import LinearColorMapper, ColorBar, CustomJS, mappers
from bokeh import events
import bokeh.palettes
from osgeo import gdal


In [0]:
def load_data():
  !pip3 install PyDrive
  import os
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials  
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  
  download = drive.CreateFile({'id': '1glwgYMyMr2P22euPjFtcIgCd8UggpQc-'})
  download.GetContentFile('S2A_MSIL1C_20180731T102021_N0206_R065_T33UUB_20180731T123616.zip')
  !ls
  
  import zipfile
  zip_ref = zipfile.ZipFile('S2A_MSIL1C_20180731T102021_N0206_R065_T33UUB_20180731T123616.zip')
  zip_ref.extractall()
  zip_ref.close()

In [0]:
load_data()

 `get_ndvi()` beräknar NDVI givet en satellitbildsfil och `reduce_array()` reducerar arrays genom att ta medelvärdet av $n \times n$-rutor.

In [0]:
def get_ndvi(file_name):
  sds = gdal.Open(file_name + "/MTD_MSIL1C.xml").GetSubDatasets()
  bands = gdal.Open(sds[0][0]).ReadAsArray()
  red = bands[2].astype(float)
  nir = bands[3].astype(float)
  print(red)
  print(nir)
  return np.divide((nir-red),(nir+red),out=np.zeros_like(red), where=(nir+red)!=0)
  

In [0]:
def reduce_array(n, arr):
    out = []
    r = arr.shape[0]
    c = arr.shape[1]
    for i in range((r + n-1)//n):
        out.append([])
        for j in range((c+n-1)//n):
            summa = 0
            num = 0
            for di in range(n):
                for dj in range(n):
                    if(i*n + di >= r or j*n + dj >= c):
                        continue
                    summa += arr[i*n+di][j*n+dj]
                    num += 1
            out[-1].append(summa/num)
    return np.array(out, dtype="float")

Beräkna NDVI:

In [0]:
ndvi = get_ndvi("S2A_MSIL1C_20180731T102021_N0206_R065_T33UUB_20180731T123616.SAFE")

Beskär bilden så vi bara får med bokskogen, och vänd på den så den passar med bokeh:

In [0]:
ndvi_cut = ndvi[3800:5500, 8300:10000]
ndvi_cut = ndvi_cut[::-1]

Lägger till en vit ruta nere i vänstra hörnet som referens. Rutan är 100 pixlar, vilket motsvarar 1 km i verkligheten.

In [0]:
def add_100_pixel_square(arr):
  ret = np.copy(arr)
  for i in range(100):
    for j in range(100):
      ret[i + 50][j + 50] = 0
  return ret

Ta fram bilden med en kvadrat i hörnet och plotta den samt lägg till en callback när man klickar på plotten: 

In [0]:
ndvi_square = add_100_pixel_square(ndvi_cut)

In [0]:
color_mapper = mappers.LinearColorMapper(palette=bokeh.palettes.PiYG[11][::-1], low=-1, high=1)
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12)
p = figure(title="NDVI", plot_height=800, plot_width=800, x_range=[-0.1, 1.1], y_range=[-0.1, 1.1], toolbar_location="below")
r = p.image(image=[ndvi_square], x=0, y=0, dw=1, dh=1, color_mapper=color_mapper)
p.add_layout(color_bar, "right")

callback = CustomJS(args=dict(source=ndvi_square), code = """
var x = cb_obj.x
var y = cb_obj.y
console.log('x: ' + cb_obj.x + ' y: ' cb_obj.y)
""")

p.js_on_event(events.Tap, callback)


In [0]:
output_notebook()
show(p)